In [ ]:
from geopy.distance import geodesic
import random

# Definir el punto fijo en el centro de la Ciudad de México
origin = (19.432608, -99.133209)  # Coordenadas aproximadas del Zócalo

# Generar un punto aleatorio a una distancia de 2 km del punto fijo
def random_point_2km_away(center, distance_km):
    angle = random.uniform(0, 360)
    destination = geodesic(kilometers=distance_km).destination(center, angle)
    return (destination.latitude, destination.longitude)

destination = random_point_2km_away(origin, 2)

print("Punto fijo (centro de la Ciudad de México):", origin)
print("Punto aleatorio a 2 km:", destination)

In [ ]:
import osmnx as ox
import geopandas as gpd

# Cargar la red de vías desde el archivo cache_MexicoCity_walk.graphml
graph = ox.load_graphml('cache_MexicoCity_walk.graphml')

# Cargar los datos de crímenes desde el archivo crimenes.geojson
crimenes = gpd.read_file('crimes.geojson')

# Mostrar información básica de los datos cargados
print(graph)
print(crimenes.head())

In [ ]:
# Re-project geometries to a projected CRS (e.g., UTM zone 14N for Mexico City)
crimenes = crimenes.to_crs(epsg=32614)

# Crear buffers de 100 metros alrededor de cada punto
crimenes['buffer'] = crimenes.geometry.buffer(100)

# Mostrar los primeros registros con los buffers
print(crimenes[['delito', 'buffer']].head())

In [ ]:
import folium
from folium.plugins import MarkerCluster

# Create a base map centered on Mexico City
map_center = [19.38, -99.15]  # Approximate center of Mexico City
crime_map = folium.Map(location=map_center, zoom_start=12, tiles='OpenStreetMap')

# Define a color mapping based on severity
severity_colors = {
    'Alto': 'red',
    'Medio': 'orange',
    'Bajo': 'blue'
}

# Create a copy of the crimenes dataframe and convert it to WGS84 (EPSG:4326)
crimenes_wgs84 = crimenes.copy()
crimenes_wgs84 = crimenes_wgs84.to_crs(epsg=4326)

# Convert the buffer geometries to WGS84
buffer_gdf = gpd.GeoDataFrame(crimenes[['delito']], geometry=crimenes['buffer'], crs=crimenes.crs)
buffer_gdf = buffer_gdf.to_crs(epsg=4326)

# Add crime buffers to the map
for idx, row in buffer_gdf.iterrows():
    # Create a popup with crime information
    popup_text = f"""
    <b>Type:</b> {crimenes_wgs84.iloc[idx]['delito']}<br>
    """

    # Choose color based on severity (assuming you have a severity column or will add one)
    color = 'blue'  # Default color if severity not available
    if 'severity' in crimenes_wgs84.columns:
        color = severity_colors.get(crimenes_wgs84.iloc[idx]['severity'], 'gray')

    # Add the buffer as a GeoJson polygon
    folium.GeoJson(
        row['geometry'],
        style_function=lambda x, color=color: {
            'fillColor': color,
            'color': color,
            'weight': 1,
            'fillOpacity': 0.5
        },
        popup=folium.Popup(popup_text, max_width=300)
    ).add_to(crime_map)

# Add legend
legend_html = '''
<div style="position: fixed; 
     bottom: 50px; right: 50px; width: 150px; height: 120px; 
     border:2px solid grey; z-index:9999; font-size:14px;
     background-color: white; padding: 10px;
     ">
     <b>Crime Severity</b><br>
     <i class="fa fa-circle" style="color:red"></i> Alto<br>
     <i class="fa fa-circle" style="color:orange"></i> Medio<br>
     <i class="fa fa-circle" style="color:blue"></i> Bajo<br>
</div>
'''
crime_map.get_root().html.add_child(folium.Element(legend_html))

# Add title
title_html = '''
<h3 align="center" style="font-size:16px"><b>Crime Buffer Zones Map (100m)</b></h3>
'''
crime_map.get_root().html.add_child(folium.Element(title_html))

# Display the map
crime_map


In [ ]:
# Encontrar los nodos más cercanos al origen y destino
origin_node = ox.distance.nearest_nodes(graph, origin[1], origin[0])
destination_node = ox.distance.nearest_nodes(graph, destination[1], destination[0])

# Calcular la ruta más corta
shortest_route = ox.shortest_path(graph, origin_node, destination_node, weight='length')

# Extraer las coordenadas de la ruta
route_coords = [(graph.nodes[node]['y'], graph.nodes[node]['x']) for node in shortest_route]

# Añadir la ruta al mapa con un color más llamativo y mayor grosor
folium.PolyLine(route_coords, color='red', weight=8, opacity=0.8).add_to(crime_map)

# Mostrar el mapa con la ruta
crime_map